In [26]:
import warnings
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model

warnings.filterwarnings('ignore')

In [27]:
# load all pickle files
model = load_model(r'..\pickle_files\model.h5')

with open(r'..\pickle_files\ohe_encoder_geography.pkl','rb') as file_obj:
    ohe_encoder_geo = pickle.load(file_obj)

with open(r'..\pickle_files\label_encoder_gender.pkl','rb') as file_obj:
    label_encoder_gender = pickle.load(file_obj)

with open(r'..\pickle_files\scaler.pkl','rb') as file_obj:
    scaler = pickle.load(file_obj)


In [28]:
# example i/p:

input_data = {
    'CreditScore': [600],
    'Geography': ['France'],
    'Gender': ['Male'],
    'Age': [40],
    'Tenure': [3],
    'Balance': [60000],
    'NumOfProducts': [2],
    'HasCrCard': [1],
    'IsActiveMember': [1],
    'EstimatedSalary': [50000]
}

In [29]:
df = pd.DataFrame(input_data)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [30]:
df['Gender'] = label_encoder_gender.transform(df['Gender'])
data = ohe_encoder_geo.transform(df[['Geography']])
df2 = pd.DataFrame(data,columns=ohe_encoder_geo.get_feature_names_out(['Geography']))
df = pd.concat([df,df2],axis=1)
df.drop('Geography',axis=1,inplace=True)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [31]:
df = scaler.transform(df)
pred = model.predict(df)
pred_prob = pred[0][0]

if pred_prob > 0.5:
    print('The customer is likely to churn')
else:
    print("The customer is not likely to churn")

1/1 [==============================] - 0s 47ms/step
The customer is not likely to churn
